In [1]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [1]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns

# Updating Tables

In [3]:
FLAT_FILE_DIR = "flat_files/"

full_table = pd.read_csv(FLAT_FILE_DIR + "full_table.csv")
event_table = pd.read_csv(FLAT_FILE_DIR + "event_table.csv")
deck_list_table = pd.read_csv(FLAT_FILE_DIR + "deck_list_table.csv")
pilot_table = pd.read_csv(FLAT_FILE_DIR + "pilot_table.csv")
deck_table = pd.read_csv(FLAT_FILE_DIR + "deck_table.csv")
card_table = pd.read_csv(FLAT_FILE_DIR + "card_table.csv")

updater = m.Updater()
updater.update_cards()

# Cleaning

New decks need to be checked for misformed codes & missing archetypes/categories

In [4]:
for idx, row in deck_table.iterrows():
    print(row["name"])

Gruul Aggro
UW Control
Red Deck Wins
Rakdos Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Rakdos Aggro
Boros Cycling Aggro
Gruul Aggro
Gruul Aggro
Rakdos Control
Rakdos Control
Gruul Aggro
Gruul Aggro
Mono Black Aggro
Gruul Aggro
Selesnya Aggro
Dimir Rogue
Gruul Aggro
Dimir Flash
Gruul Aggro
Gruul Aggro
Rakdos Aggro
UW Control
Rakdos Aggro
Rakdos Control
Rakdos Aggro
Abzan DOOM Yorion
Gruul Aggro
Dimir Mill
Gruul Aggro
Gruul Aggro
Rakdos Control
Gruul Aggro
Gruul Aggro
Gruul Aggro
UW Control
Izzet Control
Sultai Mutate Aggro
Izzet Control Kick
Mono Black Aggro
Dimir Rogue
Rakdos Aggro
Dimir Mill
Esper Yorion
Gruul Aggro
Gruul Aggro
Mono Blue Aggro
Rakdos Aggro
Gruul Aggro
Gruul Aggro
Selesnya Aggro
UW Yorion
Gruul Aggro
Gruul Aggro
Dimir Mill
Mono Black Aggro
Mono Green Aggro
Red Deck Wins
Red Deck Wins
UB Mill
Rakdos Aggro
Rakdos Aggro
UB Control
UB Mill
UB Mill
Gruul Aggro
Gruul Adventure Aggro
Gruul Aggro
UB Control
Selesnya Aggro
UB Mill
Rakdos Aggro
Gruul Aggro
Mono Black Aggro
Esper 

Gruul Aggro
Golgari Aggro
Gruul Aggro
Gruul Aggro
Mono Green Aggro
Golgari Aggro
Rakdos Aggro
Selesnya Aggro
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Dimir Mill
Gruul Adventure Aggro
Gruul Aggro
UW Yorion
Temur Ramp Ultimatum 	
Esper DOOM Yorion
Dimir Mill
Gruul Aggro
Golgari Aggro
Gruul Aggro
Gruul Aggro
Mono Green Aggro
UW Control
Mono Green Aggro
Gruul Aggro
Dimir Aggro
Temur Ramp Ultimatum
UB Mill
Esper DOOM Yorion
Gruul Aggro
Mono Red Control
Red Deck Wins
Dimir Rogue
Gruul Aggro
Esper DOOM Yorion
Gruul Aggro
Gruul Adventure Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Esper DOOM Yorion
Esper DOOM Yorion
Gruul Adventure Aggro
Gruul Aggro
Dimir Rogue
Dimir Rogue
Gruul Aggro
Selesnya Aggro
Esper DOOM Yorion
Gruul Aggro
Jund
Temur Ultimatum
Gruul Adventure Aggro
Gruul Aggro
Mono Red Control
UB Control
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Mono Green Aggro
Esper DOOM Yorion
Esper DOOM Yorion
UB Control
UB Control
Golgari Aggro
Gruul Aggro
Temur Ramp Ultimatum 	
Mono Green Aggro
Bant

Gruul Aggro
Jund
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Gruul Aggro
Rakdos Control
Temur Ultimatum
Sultai Control
Dimir Mill
Esper DOOM Yorion
Boros Control
Gruul Aggro
Gruul Aggro
Dimir Mill
Mono Green Aggro
Rakdos Aggro
Temur Ramp Ultimatum
Weenie White 
Mono Green Aggro
Gruul Aggro
Temur Ramp Ultimatum
Dimir Control
Gruul Aggro
Mono Green Aggro
Temur Ultimatum
Red Deck Wins
Dimir Control
Mono Green Aggro
Mono Red Control
Dimir Rogue
Boros Cycling Aggro
Gruul Aggro
Dimir Mill
Dimir Mill
Gruul Aggro
Dimir Yorion
Gruul Aggro
Gruul Aggro
Mono Green Aggro
Dimir Mill
Esper DOOM Yorion
Gruul Aggro
Temur Ultimatum
Red Deck Wins
Temur Ultimatum
Dimir Control
Red Deck Wins
5c Sanctum
Mono Black Aggro
Rakdos Aggro
UW Control
Mono Green Aggro
Gruul Aggro
Dimir Mill
Dimir Control
Esper DOOM Yorion
Esper DOOM Yorion
Gruul Aggro
Red Deck Wins
Simic Aggro
Obosh Temur Ultimatum
Dimir Rogue
Rakdos Sacrifice
Winota
Boros Cycling Aggro
Dimir Mill
Mono Blue Control
UR Aggro
Rakdos Aggro
Gruul Aggro
M

In [5]:
deck_table.loc[deck_table.archetype.isna()].empty

True

In [6]:
deck_table.loc[deck_table.category.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [7]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

In [8]:
unmatched

{nan}

In [9]:
standard_sets = "eld|thb|znr|iko|khm|m21"
names = full_table[
    (~full_table["cardId"].str.contains(standard_sets, regex=True))
    & (full_table["date"] >= "2020-09-29")
][["cardId", "name"]].drop_duplicates().reset_index(drop=True)["name"]

In [10]:
names

0    Midnight Reaper
1             Murder
Name: name, dtype: object

In [11]:
if (names == pd.Series(["Midnight Reaper", "Murder"])).all():
    print("do a thing")

do a thing


In [12]:
deck_table[
    deck_table["deckId"].isin(
        full_table[
            full_table["cardId"] == "101m13"
        ]
        ["deckId"]
    )
]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category
552,553,102,479,?e=27633&d=419513&f=ST,Jund,2,jund,jund


In [13]:
try:
    has_points = deck_table[deck_table["rank"].str.contains("pts")]
except AttributeError:
    has_points = "No Points"
has_points

'No Points'